# 2.1 Debugging

In [1]:
import os
os.chdir('../home')

% ## Sections
% - Introduction to debugging: importance and basic concepts.
% - Configuration of the debugging environment in Visual Studio Code.
- Use of breakpoints to halt execution.
- Types of breakpoints: conditional and exception breakpoints.
- Inspection of variables and expressions during execution.
- Utilization of the debugging control panel.
- Step-by-step execution: step into, step over, and step out.
- Visualization tools: watch, variables, and call stack.
- Logging usage for debugging.
- Remote debugging and debugging in virtual environments.
- Using the justMyCode option to understand the behavior of third-party libraries. 
% https://github.com/scikit-learn/scikit-learn/blob/6eff1757e/sklearn/linear_model/_coordinate_descent.py#L710
% In class, I can also show scikit-learn's code for the ElasticNet, in particular fit and from fit deep dive into
% _validate_data.

# Mastering Debugging in Python with Visual Studio Code

## Motivation

Debugging is not merely about fixing errors; it's about understanding the behavior of your code under various conditions, ensuring reliability, and improving performance. Effective debugging can significantly reduce development time, enhance code quality, and create more robust applications.

Imagine you're working on a Python project that processes data from an online store. Your task is to write a script that calculates the total sales for each product category. However, after trying to run your script, you notice that is throwing an error. You've checked your code multiple times, but the mistake remains elusive.

In [2]:
def calculate_total_sales(sales_data):
    total_sales = 0
    for category, sales in sales_data.items():
        total_sales += sales
    return total_sales

sales_data = {"electronics": 10000, "books": 5000, "clothing": "7000", "toys": 3000},
total_sales = calculate_total_sales(sales_data)
print(f"Total sales: ${total_sales}")

AttributeError: 'tuple' object has no attribute 'items'

### Setting Up and Running the Debugger

1. **Open your Python script in VS Code** and make sure you have the Python extension installed.
2. **Open the Debug view** by clicking on the Run and Debug view icon on the left sidebar or pressing `Ctrl+Shift+D`.
4. **Start debugging** by selecting the appropriate Python file configuration and pressing the green play button or pressing `F5`. Make sure the "Uncaught Exceptions" option is ticked in the Breakpoints panel.

### Observing the Exception and Inspecting Variables

As the debugger runs, it will halt execution when it encounters the uncaught TypeError exception due to attempting to add an integer to a string. This halting point allows us to inspect the program's state just before the exception occurred.

- **Inspect the `sales_data` variable** in the Variables panel to notice that the value for "clothing" is a string, `"7000"`, instead of an integer.

### Setting a Breakpoint and Watching Variables

With the knowledge that our program halts due to a TypeError, we'll focus on the calculation of `total_sales`:

1. **Add a breakpoint** at the `total_sales += sales` line.
2. **Add `total_sales` and `sales` to the Watch panel** to monitor their values in real-time.
3. **Restart the debugger** and observe as it stops at our breakpoint before the exception occurs.
4. **Step through the for loop** using the "Step Over" button (or pressing `F10`) to iterate through each category and sale.

### Identifying and testing a solution for the Issue

As you step through each iteration, when reaching "clothing", you'll notice in the Watch panel that `sales` holds the string value `"7000"`. This is the moment of realization that the data type inconsistency is causing our issue.

1. **Modify the `sales_data` directly in the watch panel** with a secondary click and selecting "Change Value" to correct the value for "clothing" from `"7000"` to `7000` (an integer).
2. **Continue execution** by pressing the "Continue" button (or pressing `F5`) in the debugger.
3. Observe that the debugger now completes the execution without halting on an exception, and the correct total sales value is printed to the console.

## Other Types of Breakpoints
In our first example, we used an uncaught exception breakpoint to halt the execution of our program. Let's now use our example to explore other types of breakpoints.

### **Conditional Breakpoints**

Pause execution when a specified condition is true.

1. Open your code in Visual Studio Code and navigate to the `for` loop line.
2. Right-click on the left margin and select "Add Conditional Breakpoint."
3. Enter `sales == 5000` as the condition. This setup ensures the debugger halts for the "books" category, allowing inspection of variables at that point.

### **Hit Count Breakpoints**

Trigger after the breakpoint has been hit a specified number of times.

1. On the same line as before, add a breakpoint.
2. Right-click on the breakpoint and select "Edit Breakpoint" -> "Hit Count."
3. Specify the hit count as `3`. The debugger will now pause execution the third time it reaches this line, useful for examining the accumulated `total_sales` after several iterations.

### **Logpoints**

Log messages to the console without stopping the program.

1. Right-click on the left margin next to the loop and select "Add Logpoint."
2. Type `Processing {category}: {sales}` in the message field. This action allows real-time monitoring of which items are being processed and their respective sales without halting the program.

### **Function Breakpoints**

Halt execution when entering a specified function.

1. Go to the Run and Debug view in Visual Studio Code.
2. Click on "Add Function Breakpoint" and type the name of the function, `calculate_total_sales`.
3. When you run the debugger, it will pause every time `calculate_total_sales` is invoked, allowing you to inspect the initial state of the function and its parameters.

### **Exception Breakpoints**

Pause execution when exceptions are thrown.

- **For Raised Exceptions**:
    1. In the Debug view, under Breakpoints, ensure that "All Exceptions" is checked.
    2. If an exception is raised within `calculate_total_sales` or its call, the debugger will halt, enabling you to inspect the cause and context of the exception.

- **For User-Uncaught Exceptions**:
    1. Similarly, check "Uncaught Exceptions" to focus on exceptions that escape your error handling. This is particularly useful for identifying and debugging unhandled exceptions that could cause unexpected behavior or application crashes.


### Function Breakpoints and Debugging Inside a Third-Party Library

Suppose we are working on a Python project that uses pandas for data manipulation and we have the following code:

In [3]:
import pandas as pd
s = pd.Series([1, 2, 3, 4])
s.add_suffix('_item')

/tmp/ipykernel_1770498/179978455.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


0_item    1
1_item    2
2_item    3
3_item    4
dtype: int64

We want to understand how the `add_suffix` method works internally, in particular, we would like to investigate an issue that occurs with `_get_axis_name` which is another method that is used by `add_suffix`. To do this, we'll configure our debugger to step into the pandas library code and break when it reaches the `_get_axis_name` function.

#### Step 1: Configuring the Debugger

1. **Open VS Code** and navigate to your Python script.
2. **Open the `launch.json` file** in the `.vscode` folder. If it doesn't exist, create it by going to the Run and Debug view, selecting "create a launch.json file", and then choosing a Python configuration.
3. **Modify the `launch.json` file** to include the `justMyCode` option set to `false`. This tells the debugger to include third-party library code in the debugging session.

```json
{
    "version": "0.2.0",
    "configurations": [
        {
            "name": "Python: Current File",
            "type": "python",
            "request": "launch",
            "program": "${file}",
            "justMyCode": false
        }
    ]
}
```

#### Step 2: Setting a Function Breakpoint

1. **Go to the Run and Debug view** in VS Code.
2. **Open the Breakpoints pane** and find the option to add a function breakpoint. Click on "Add Function Breakpoint".
3. **Type the name of the function** you want to break on. In this case, enter `_get_axis_name`.
4. **Start the debugger** by selecting the appropriate configuration for the current file and pressing the green play button or pressing `F5`.

#### What to Expect

When you run the debugger with these settings and execute your script, the debugger will pause execution when it reaches the `_get_axis_name` function call within the pandas library. This allows you to:

- Step through the pandas library code to see how the `_get_axis_name` function is implemented.
- Inspect variables and understand the internal workings of the function.
- Gain insights into the behavior of third-party libraries, which can be invaluable for debugging complex issues or enhancing your understanding of these libraries.

#### Note

Remember that stepping into third-party library code can sometimes be overwhelming due to the complexity and volume of the code. However, it can also be a powerful way to learn more about these libraries and to troubleshoot issues more effectively or even adapt them to your specific needs.


## References
https://code.visualstudio.com/docs/python/python-quick-start
https://code.visualstudio.com/docs/python/debugging
https://code.visualstudio.com/docs/editor/debugging